In [14]:
PAGE_SIZE = 100

In [2]:
from urllib.request import urlopen
import json

#get number of pages
count_response = urlopen('https://yandex.ru/znatoki/api/v2/questions?page_size=1')
count = json.loads(count_response.read())['count']

In [20]:
def process_question(question):
    data = {key: question[key] for key in ('id', 'created', 'tags', 'title', 'text', 'updated', 'votes', 'answersCount', 'subscribersCount')}
    if question['previewAnswer']:
        data['best_answer_text'] = question['previewAnswer']['plainText']
        data['best_answer_votes'] = question['previewAnswer']['votes']
        data['best_answer_id'] = question['previewAnswer']['id']
        data['best_answer_created'] = question['previewAnswer']['created']
        data['best_answer_author_id'] = question['previewAnswer']['author']['id']
    return data

In [21]:
#get all questions
all_questions = {}
with open('questions.jsonl', 'w') as outfile:
    for i in range(0,count // PAGE_SIZE):
        questions_response = urlopen('https://yandex.ru/znatoki/api/v2/questions?page={}&page_size=100'.format(i))
        questions_results = json.loads(questions_response.read().decode('utf-8'))['results']
        for j in range(0, len(questions_results)):
            json.dump(process_question(questions_results[j]), outfile)
            outfile.write('\n')

In [23]:
import zipfile
with zipfile.ZipFile('questions.zip', 'w') as myzip:
    myzip.write('questions.jsonl', compress_type=zipfile.ZIP_DEFLATED)

In [22]:
import pandas as pd
import pandas_profiling
import numpy as np

In [14]:
from pandas.io.json import json_normalize

questions_df = pd.DataFrame()
questions = []
with open('questions.jsonl', 'r') as f:
    for question in f:
        json_normalized_question = json_normalize(json.loads(question))
        questions.append(json_normalized_question)
questions_df = questions_df.append(questions)

In [18]:
questions_df.head()
# questions_df.shape

,answersCount,created,id,tags,text,title,updated,votes
0,36,2018-01-14T19:16:55.558618+00:00,c438a923-f2b7-406c-aa28-3307635307db,"[животные, кошки]",,"Как объяснить коту, что он не прав?",2018-11-26T14:43:15.256540+00:00,30
0,24,2018-06-13T10:33:09.995850+00:00,5daf1c48-668a-493d-b6dd-3962b8d90824,"[автомобили, ремонт автомобилей]",,Как убрать ржавчину с кузова автомобиля своими...,2018-11-24T22:25:35.113159+00:00,26
0,99,2018-09-05T12:02:30.232305+00:00,277d8b8a-8a8d-40d0-afc1-771069864654,[],,"Как убрать блок в Яндекс поиске ""Вам может быт...",2018-11-20T14:42:14.522666+00:00,119
0,35,2018-09-09T22:33:28.971831+00:00,ef9944df-e24d-487c-88a4-d74e50a1be68,[],,Как отключить блок «Вам может быть интересно» ...,2018-11-20T14:20:00.728582+00:00,65
0,5,2018-07-12T23:22:27.977448+00:00,8fd3d0f8-7871-4d65-b42f-abfb06f532a3,"[квантовая механика, физика]",,Почему электроны не падают на ядро?,2018-11-26T16:27:22.102208+00:00,11


In [13]:
#TODO processing  dataframe here     
questions_df['created', 'updated', 'best_answer_created'] = questions_df['created', 'updated', 'best_answer_created'].apply(lambda x: x[0:10])

'2018-07-22'

In [16]:
pandas_profiling.ProfileReport(questions_df)

Number of variables,9
Number of observations,334200
Total Missing (%),0.0%
Total size in memory,22.9 MiB
Average record size in memory,72.0 B
Numeric,2
Categorical,4
Boolean,0
Date,0
Text (Unique),1
Rejected,1


In [ ]:
import collections
tags = []
for question_tags in questions_df['tags']:
    tags.append(question_tags)
c = collections.Counter(tags)
c.most_common()

In [17]:
pfr = pandas_profiling.ProfileReport(questions_df)
pfr.to_file("znatoki_report.html")